# Introducción a Redes Convolucionales

## Convoluciones

Realizamos un análisis intuitivo del efecto de las convoluciones aplicando filtros sobre una imagen.

Importamos algunos módulos y hacemos configuraciones para los gráficos del cuaderno:

In [ ]:
from scipy.ndimage import convolve #Usamos la clase convolve para implementar una convolución
import numpy as np
import matplotlib.pyplot as plt
% matplotlib inline
plt.rcParams["savefig.dpi"] = 300
plt.rcParams["savefig.bbox"] = "tight"
plt.rcParams["figure.figsize"] = (16,9)
np.set_printoptions(precision=3, suppress=True)
import pandas as pd

También preparamos una función para visualizar mejor los gráficos de pérdida y precisión que utilizaremos luego:

In [ ]:
def plot_history(logger):
    df = pd.DataFrame(logger.history)
    df[['accuracy', 'val_accuracy']].plot()
    plt.ylabel("precisión")
    df[['loss', 'val_loss']].plot(linestyle='--', ax=plt.twinx())
    plt.ylabel("pérdida")

Vamos a generar los datos para hacer un difuminado Gaussiano:

In [ ]:
rng = np.random.RandomState(2)
signal = np.cumsum(rng.normal(size=200))
plt.plot(signal)

Ahora generamos la función filtro gausiano y la mostramos:

In [ ]:
gaussian_filter = np.exp(-np.linspace(-2, 2, 15) ** 2)
gaussian_filter /= gaussian_filter.sum()
plt.plot(gaussian_filter)
gaussian_filter

Aplicamos la convolución (composición) de las dos funciones llamando a la clase `convolve`:

In [ ]:
plt.plot(signal)
plt.plot(convolve(signal, gaussian_filter))

Descargamos una imagen para mostrar cuál es el efecto de un filtro gaussiano sobre una imagen:

In [ ]:
!curl -o Caballero.jpg "https://upload.wikimedia.org/wikipedia/commons/thumb/f/f1/El_caballero_de_la_mano_en_el_pecho.jpg/600px-El_caballero_de_la_mano_en_el_pecho.jpg"

Una vez que la imagen está descargada en el sistema de ficheros local, la importamos y la mostramos:

In [ ]:
import imageio
image = imageio.imread('Caballero.jpg')
plt.imshow(image)

Usamos el filtro gaussiano unidimensional que creamos más arriba para crear un filtro gaussiano bidimensional y mostramos el aspecto que tiene (esperamos una función discretizada, ya que la definimos así más arriba):

In [ ]:
gaussian_2d = gaussian_filter * gaussian_filter[:, np.newaxis]
plt.matshow(gaussian_2d)

Ahora hacemos la convolución del filtro sobre la imagen que hemos cargado, para apreciar el efecto que tiene, que debería ser un difuminado:

In [ ]:
out = convolve(image, gaussian_2d[:, :, np.newaxis])
plt.imshow(out)

Vamos a transformar la imagen a escala de grises. `image` es un array numpy, para hacer la escala de grises computamos la media entre los tres canales de color de la imagen, que están expresados en la tercera dimensión (el eje 2 del array):

In [ ]:
gray_image = image.mean(axis=2)
plt.imshow(gray_image, cmap="gray")

Componemos ahora un filtro similar al Sobel, para ellos componemos la función que expresa el fitro gaussiano con una matriz [-1,1]:

In [ ]:
gradient_2d = convolve(gaussian_2d, [[-1, 1]])
plt.imshow(gradient_2d)

Aplicamos este filtro, que lo que va a hacer es resaltar el gradiente en la dirección del eje x de la imagen, mostrando así más intensidad en líneas verticales:

In [ ]:
edges = convolve(gray_image, gradient_2d)
plt.imshow(edges, cmap="gray")

## CNNs con Keras

Vamos a ver cómo se configuran redes convolucionales (CNNs, de *Convolutional Neural Networks*) con Keras.

Empezamos con la preparación de datos. Lo primero es el preprocesado de los datos.

In [ ]:
from keras.datasets import mnist     # MNIST dataset incluido en Keras
from keras.models import Sequential  # Tipo de modelo a utilizar
from keras.utils import np_utils

from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten # Tipos de capas que usaremos en nuestro modelo

In [ ]:
# Los datos MNIST están repartidas entre 60000 imágenes de 28 x 28 píxeles
# y 10000 imágenes de 28 x 28 píxeles
img_rows, img_cols = 28, 28

# Separamos los datos en train y test
(X_train, y_train), (X_test, y_test) = mnist.load_data()

print(f"X_train shape: {X_train.shape}")
print(f"y_train shape, {y_train.shape}")
print(f"X_test shape, {X_test.shape}")
print(f"y_test shape, {y_test.shape}")

In [ ]:
# Pasamos de enteros a números en punto flontate de 32 bits
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

# Normalizamos cada valor de cada pixel para cada vector de entrada
X_train /= 255
X_test /= 255

Hacemos un *one hot encoding* de las las etiquetas:

In [ ]:
num_classes = 10
y_train = np_utils.to_categorical(y_train, num_classes)
y_test = np_utils.to_categorical(y_test, num_classes)

Configuramos algunos de los hiperparámetros de la red:

In [ ]:
batch_size = 128
epochs = 12

Modificamos las dimensiones de las muestras para que sean 4D, tal y como esperan las capas convolucionales en la entrada en Keras. La cuarta dimensión es el canal correspondiente a la imagen en blanco y negro (si fuesen imágenes en color, tendríamos tres canales):

In [ ]:
X_train = X_train.reshape(X_train.shape[0], img_rows, img_cols, 1)
X_test = X_test.reshape(X_test.shape[0], img_rows, img_cols, 1)
input_shape = (img_rows, img_cols, 1)
print(f"X_train shape: {X_train.shape}")
print(f"X_test shape, {X_test.shape}")

Configuramos una red neuronal. Es una red pequeña porque no queremos gastar mucho tiempo en el entrenamiento. Vamos a utilizar un modelo secuencia, como hicimos con las MLPs que configuramos con Keras anteriormente:

In [ ]:
cnn = Sequential()

La capa `Conv2D` acepta varios parámetros. El primero, el número 32 es el número de feature maps que vamos a generar, y el siguiente es el tamaño del kernel (filtro). Así pues, esta primera capa va a aprender 32 kernels de tamaño 3x3.

La dimensión de entrada es la que configuramos antes, 28x28x1:

In [ ]:
cnn.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))


La salida de esta capa tendrá una dimensión de 26x26x32, porque estamos generando 32 convoluciones con kernels de 3x3, que está haciendo una convolución válida. Como el filtro es de 3x3, significa que estamos perdiendo un pixel por cada lado en la matrix inicial de 28x28 que es cada imagen de entrada (la fórmula es que para filtros impares, siempre perdemos un número de pixels = anchura del filtro -1 = 3 - 1 = 2 píxels que perdemos).

Hacemos un max pooling con un tamaño de 2x2:

In [ ]:
cnn.add(MaxPooling2D(pool_size=(2, 2)))

Al hacer el max pooling 2x2, la resolución se reduce a la mitad, y de ahí que tengamos 13x13.

Añadimos una convolución y un max pooling adicional:

In [ ]:
cnn.add(Conv2D(32, (3, 3), activation='relu'))
cnn.add(MaxPooling2D(pool_size=(2, 2)))

Añadimos una capa de flatten, que va a convertir básicamente las 32 matrices bidimensionales que vamos arrastrando en un gran vector que podrá ser la entrada para una capa densa, sobre la que finalmente aplicaremos una activación softmax para que nos de las clases que estamos buscando:

In [ ]:
cnn.add(Flatten())
cnn.add(Dense(64, activation='relu'))
cnn.add(Dense(num_classes, activation='softmax'))

Si pedimos un resumen de la red, podemos analizar las dimensiones de cada capa y ver que todo tiene sentido:

In [ ]:
cnn.summary()

Lo interesante de ver aquí es que el número de parámetros entrenables que tenemos en una red convolucional es mucho menor que el número que obteníamos en una red densa convencional como con la que estuvimos trabajando previamente cuando introdujimos Keras.

En las primeras capas convolucionales tenemos muy pocos parámetros comparado con el número de parámetros que tenemos en una capa densa.

Sin embargo, habitualmente en las CNNs el tamaño de las activaciones es más grande que en las redes convencionales (y, como hemos visto, el número de pesos es más pequeño).

Otra cosas interesante es que la segunda capa convolucional tiene muchos más parámetros. Esto es debido a que cada 32 filtros de 3x3 tienen que aplicarse a las 32 imágenes de salida de la capa previa que son resultado a su vez de la aplicación de 32 kernels de 3x3. Así pues, $32(32\cdot(3\cdot3)) +32 = 9248$, que coincide con el número de parámetros que deberíamos ver en esa capa.

Compilamos y entrenamos, utilzando como optimizador `adam`, y usando 20 épocas y reservando un 10% de los datos para validación 

In [ ]:
cnn.compile("adam", "categorical_crossentropy", metrics=['accuracy'])
history_cnn = cnn.fit(X_train_images, y_train,
                      batch_size=128, epochs=20, verbose=1, validation_split=.1)

El modelo está entrenado, que quiere decir que tiene todos los filtros y los pesos aprendidos. Si queremos utilizarlo cuando cerremos el entorno de ejecución del cuaderno sin tener que volver a entrenar, podemos guardarlo invocando el método `save` de nuestro modelo:

In [ ]:
cnn.save('mymodel')

In [ ]:
plot_history(history_cnn)

Si evaluamos el modelo, vemos que la precisión de test es mejor que lo que obtuvimos antes con una red convencional, pese a que tiene menos parámetros.

In [ ]:
cnn.evaluate(X_test_images, y_test)

In [ ]:
df = pd.DataFrame(history_cnn.history)
df[['accuracy', 'val_accuracy']].plot()
plt.ylabel("precisión")
plt.ylim(.9, 1)

Vamos a construir una red algo más pequeña para visualizar los filtros y los resultados de aplicar la convolución (lo que serían nuestros feature maps):

In [ ]:
cnn_small = Sequential()
cnn_small.add(Conv2D(8, kernel_size=(3, 3),
              activation='relu',
              input_shape=input_shape))
cnn_small.add(MaxPooling2D(pool_size=(2, 2)))
cnn_small.add(Conv2D(8, (3, 3), activation='relu'))
cnn_small.add(MaxPooling2D(pool_size=(2, 2)))
cnn_small.add(Flatten())
cnn_small.add(Dense(64, activation='relu'))
cnn_small.add(Dense(num_classes, activation='softmax'))

In [ ]:
cnn_small.summary()

In [ ]:
cnn_small.compile("adam", "categorical_crossentropy", metrics=['accuracy'])
history_cnn_small = cnn_small.fit(X_train_images, y_train,
                      batch_size=128, epochs=10, verbose=1, validation_split=.1)

Veamos ahora cuáles son las dimensiones que estamos esperando de los diferentes filtros en las dos capas convolucionales:

In [ ]:
weights, biases = cnn_small.layers[0].get_weights()
weights2, biases2 = cnn_small.layers[2].get_weights()
print(weights.shape)
print(weights2.shape)

En la primera capa, tenemos sobre un canal de entrada filtros de 3x3 que nos dan ocho canales de salida, y en la segunda tenemos ocho canales de entrada con filtros de 3x3 que nos dan también ocho canales de salida.

Vamos a visualizar los filtros:

In [ ]:
fig, axes = plt.subplots(9, 8, figsize=(10, 8), subplot_kw={'xticks': (), 'yticks': ()})
mi, ma = weights.min(), weights.max()
for ax, weight in zip(axes[0], weights.T):
    ax.imshow(weight[0, :, :].T, vmin=mi, vmax=ma)
axes[0, 0].set_ylabel("layer1")
mi, ma = weights2.min(), weights2.max()
for i in range(1, 9):
    axes[i, 0].set_ylabel("layer3")
for ax, weight in zip(axes[1:].ravel(), weights2.reshape(3, 3, -1).T):
    ax.imshow(weight[:, :].T, vmin=mi, vmax=ma)

Tenemos 1 input channel para la primera capa convolucional con filtros de 3x3, y 8 feature maps de salida.

Tomando la primera fila, por columnas tenemos los filtros para cada uno de los feature maps se que van a generar.

Para la segunda capa convolucional, tenemos 8 canales de entrada y 8 canales de salida. Como funciona es que la segunda fila de filtros en la matriz se aplican a los resultados de los filtros de la primera fila, y se suman para darnos un feature map. Lo mismo con la tercera fila para dar otro feature map y así hasta la novena fila (8 filas correspondientes a la segunda capa), que generan en total 8 feature maps.

De esta manera es como se ve que hay más parámetros en la segunda capa convolucional, porque ahora todos los filtros tienen que procesar todos los feature maps generados en la primera capa.

In [ ]:
from keras import backend as K

get_1rd_layer_output = K.function([cnn_small.layers[0].input],
                                  [cnn_small.layers[0].output])
get_3rd_layer_output = K.function([cnn_small.layers[0].input],
                                  [cnn_small.layers[2].output])

layer1_output = get_1rd_layer_output([X_train_images[:5]])[0]
layer3_output = get_3rd_layer_output([X_train_images[:5]])[0]

In [ ]:
layer1_output.shape

In [ ]:
layer3_output.shape

Veamos ahora las activaciones de esta red con ocho feature maps y filtros de 3x3:

In [ ]:
weights, biases = cnn.layers[0].get_weights()
n_images = layer1_output.shape[0]
n_filters = layer1_output.shape[3]
fig, axes = plt.subplots(n_images * 2, n_filters + 1, figsize=(10, 8), subplot_kw={'xticks': (), 'yticks': ()})
for i in range(layer1_output.shape[0]):
    # for reach input image (= 2 rows)
    axes[2 * i, 0].imshow(X_train_images[i, :, :, 0], cmap="gray_r")
    axes[2 * i + 1, 0].set_visible(False)
    axes[2 * i, 1].set_ylabel("layer1")
    axes[2 * i + 1, 1].set_ylabel("layer3")
    for j in range(layer1_output.shape[3]):
        # for each feature map (same number in layer 1 and 3)
        axes[2 * i, j + 1].imshow(layer1_output[i, :, :, j], cmap='gray_r')
        axes[2 * i + 1, j + 1].imshow(layer3_output[i, :, :, j], cmap='gray_r')


En la primera fila, vemos los feature maps generados a partir de la imagen de la izquierda, que son los resultados una vez aplicados los ocho filtros aprendidos, de dimensiones 26x26. 

En la segunda fila se ve el resultado de la segunda capa convolucional, 8 feature maps de 11x11 obtenidos como hemos descrito antes. Estas son las activaciones en la segunda capa convolucional, después de la cual se aplican las capas densas.

Por ejemplo, puede observarse en la fila correspondiente al número 1 cómo el filtro está claramente priorizando el reconocimiento de una línea vertical ligeramente inclinada.

# Batch Normalization

Probamos el efecto de Batch Normalization en la red convolucional en este cuaderno que hemos configurado en este cuaderno, y analizamos la mejora de rendimiento que una red convolucional introduce.

Añadimos un tipo de capa más, `BatchNormalization`, configuramos nuestra red neuronal de una manera muy parecida a cómo hemos hecho antes, pero con capas de Batch Normalization después de las activaciones de las convoluciones:

In [ ]:
from keras.layers import BatchNormalization, Activation

cnn_small_bn = Sequential()
cnn_small_bn.add(Conv2D(8, kernel_size=(3, 3),
                 input_shape=input_shape))
cnn_small_bn.add(Activation("relu"))
cnn_small_bn.add(BatchNormalization())
cnn_small_bn.add(MaxPooling2D(pool_size=(2, 2)))
cnn_small_bn.add(Conv2D(8, (3, 3)))
cnn_small_bn.add(Activation("relu"))
cnn_small_bn.add(BatchNormalization())
cnn_small_bn.add(MaxPooling2D(pool_size=(2, 2)))
cnn_small_bn.add(Flatten())
cnn_small_bn.add(Dense(64, activation='relu'))
cnn_small_bn.add(Dense(num_classes, activation='softmax'))

Revisamos la arquitectura de la red que acabamos de configurar:

In [ ]:
cnn.summary()

Escribir el gradiente del minibatch a mano es muy complicado. Sin embargo, todos los frameworks de Deep Learning incorporan *AutoDiff*, que computa los gradientes por nosotros.

In [ ]:
cnn_small_bn.compile("adam", "categorical_crossentropy", metrics=['accuracy'])
history_cnn_small_bn = cnn_small_bn.fit(X_train_images, y_train,
                                        batch_size=128, epochs=10, verbose=1, validation_split=.1)

Aquí tenemos una comparación entre la precisión en training y validación entre una red convolucional pequeña con y sin batch normalization, y se puede ver que los resultados son mejores cuando incorporamos esta técnica: